In [1]:
import os

In [2]:
%pwd

'd:\\project\\chicken-fecal-classification_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\project\\chicken-fecal-classification_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path
from typing import List


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir : Path
    base_model_path : Path
    updated_base_model_path : Path
    params_image_size : List[int]
    params_learning_rate : float
    params_include_top : bool
    params_weights : str
    params_classes : int


In [6]:
from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create the directories specified in the config file
        create_directories([Path(self.config["artifacts_root"])])

    # Define any additional methods here, for example:
    def get_data_ingestion_config(self):
        # Example method to retrieve configuration details
        data_config = self.config.get("data_ingestion", {})
        return data_config


In [8]:
from pathlib import Path

class configurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ) -> None:
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(params_filepath)

        # Debug print to verify structure of self.config
        print("Loaded config:", self.config)  # Check if 'data_ingestion' and 'artifacts_root' keys exist

        # Convert `artifacts_root` to Path object before passing to `create_directories`
        create_directories([Path(self.config["artifacts_root"])])

    def get_data_ingestion_config(self) -> PrepareBaseModelConfig:

        config = self.config.prepare_base_model
        print(config)

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            params_image_size = self.param.IMAGE_SIZE,
            paramslearning_rate = self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            param_classes=  self.params.CLASSES
        )

        return prepare_base_model_config
    
    #get_prepare_base_model_config
        
        
    
        

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [10]:
import tensorflow as tf
from pathlib import Path

class PrepareBasemodel:
    def __init__(self, config):
        self.config = config  # This should be an instance of PrepareBaseModelConfig

    def get_base_model(self):
        # Access properties directly from config without attempting to iterate
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.image_size,   # Adjust this according to your PrepareBaseModelConfig attributes
            weights=self.config.weights,          # Adjust this according to your PrepareBaseModelConfig attributes
            include_top=self.config.include_top   # Adjust this according to your PrepareBaseModelConfig attributes
        )
        self.save_model(path=self.config.base_model_path, model=self.model)

    def save_model(self, path, model):
        model.save(path)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif freeze_till is not None and freeze_till > 0:
            for layer in model.layers[:freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)


In [13]:
pip install pyYAML

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from cnn_classifier.entity.config_entity import DataIngestionConfig,PrepareBaseModelConfig
from cnn_classifier.entity.config_manager import ConfigurationManager
import yaml
try:
    config = ConfigurationManager() # Initialize configuration manager
    prepare_base_model_config = config.get_prepare_base_model_config()  # Get base model config
    # Now you can use the configuration to pass into your model preparation
    prepare_base_model = PrepareBasemodel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()

except Exception as e:
    print(f"An error occurred: {e}")
    raise e


An error occurred: name 'yaml' is not defined


NameError: name 'yaml' is not defined

In [19]:
# Example usage
import yaml


try:
    # Initialize configuration manager
    config_manager = ConfigurationManager()
    
    # Get the base model configuration
    prepare_base_model_config = config_manager.get_prepare_base_model_config()
    
    # Use the configuration to prepare your base model (For example)
    print(prepare_base_model_config.root_dir)
    print(prepare_base_model_config.image_size)
    print(prepare_base_model_config.params_classes)
    
    # Add your logic to load and update the base model using the above config

except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: 'ConfigurationManager' object has no attribute 'get_prepare_base_model_config'


In [17]:
import yaml
from cnn_classifier.entity.config_manager import ConfigurationManager

try:
    config = ConfigurationManager()  # Initialize configuration manager
    prepare_base_model_config = config.get_prepare_base_model_config()  # Get base model config

    prepare_base_model = PrepareBasemodel(config=prepare_base_model_config)
    base_model = prepare_base_model.get_base_model()  # Get the base model
    print(base_model.summary())  # Optionally print the model summary to verify

except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: name 'yaml' is not defined
